In [67]:
from numpy import hstack, ones, array, mat, tile, reshape, squeeze, eye, asmatrix
from numpy.linalg import inv
from pandas import read_csv, Series 
from scipy.linalg import kron
from scipy.optimize import fmin_bfgs
import numpy as np
import statsmodels.api as sm

In [68]:
# STATIC FUNCTIONS
iteration = 0
lastValue = 0
functionCount = 0

def iter_print(params):
    global iteration, lastValue, functionCount
    iteration += 1
    print('Func value: {0:}, Iteration: {1:}, Function Count: {2:}'.format(lastValue, iteration, functionCount))

In [69]:
from statsmodels.sandbox.regression.gmm import GMM

class gmm(GMM):
    def momcond(self, params):
        endog = self.endog
        exog = self.exog
        inst = self.instrument
        
        T,N = endog.shape
        T,K = exog.shape
        beta = squeeze(array(params[:(N*K)]))
        lam = squeeze(array(params[(N*K):]))
        beta = reshape(beta,(N,K))
        lam = reshape(lam,(K,1))
        betalam = beta @ lam
        expectedRet = exog @ beta.T
        e = endog - expectedRet
        instr = tile(exog,N)
        moments1 = kron(e,ones((1,K)))
        moments1 = moments1 * instr
        moments2 = endog - betalam.T

        g = np.column_stack((moments1, moments2))
        self.moments = hstack((moments1,moments2))
        return self.moments
    
    def gmmobjective(self, params, Winv, out=False):
        global lastValue, functionCount
        endog = self.endog
        exog = self.exog
        inst = self.instrument
        T,N = endog.shape
        T,K = exog.shape
        
        moments = self.momcond(params)
        avgMoment = moments.mean(axis=0)

        J = T * mat(avgMoment) * mat(Winv) * mat(avgMoment).T
        J = J[0,0]
        lastValue = J
        functionCount += 1
        if not out:
            return J
        else:
            return J, moments
    
    def gmm_G(self, params):
        pRets = self.endog
        fRets = self.exog
        
        T,N = pRets.shape
        T,K = fRets.shape
        beta = squeeze(array(params[:(N*K)]))
        lam = squeeze(array(params[(N*K):]))
        beta = reshape(beta,(N,K))
        lam = reshape(lam,(K,1))
        G = np.zeros((N*K+K,N*K+N))
        ffp = (fRets.T @ fRets) / T
        G[:(N*K),:(N*K)]=kron(eye(N),ffp)
        G[:(N*K),(N*K):] = kron(eye(N),-lam)
        G[(N*K):,(N*K):] = -beta.T

        return G

In [70]:
# read in data
data = read_csv('EA3_data.csv')
# # filter to only results from paper
# data = data[data['Date'] < '1964']

# write date, factor, riskfree and portfolio columns
dates = data['Date'].values
factors = data[['NdrMinus','Ncf']].values*100
riskfree = data['Rf'].values*100
portfolios = data.iloc[:,6:].values*100

# T,N = portfolios.shape
# portfolios = portfolios[:,np.arange(0,N,2)]
T,N = portfolios.shape
excessRet = portfolios - np.reshape(riskfree,(T,1))
K = np.size(factors,1)

In [71]:
# generate starting values from an ols regression
betas = []
for i in range(N):
    res = sm.OLS(excessRet[:,i],sm.add_constant(factors)).fit()
    betas.append(res.params[1:])

avgReturn = excessRet.mean(axis=0)
avgReturn.shape = N,1
betas = array(betas)
res = sm.OLS(avgReturn, betas).fit()
riskPremia = res.params
riskPremia.shape = 2

# concatenate endog betas with start riskpremia
starting_vals = np.concatenate((betas.flatten(),riskPremia))

# generate a starting weight array using np.eye
starting_weights = np.eye(N*(K+1))

In [72]:
# tile factors against the length of the portfolios to get instruments
inst = tile(factors,N)

# run GMM
GMM1 = gmm(endog=excessRet, 
          exog=factors, 
          instrument=inst, 
          k_moms=27, 
          k_params=3)
res = GMM1.fit(start_params=starting_vals, inv_weights=starting_weights)
step1_vals = res.params
step1_tvals = res.tvalues
step1_pvals = res.pvalues
_, step1_moments = GMM1.gmmobjective(step1_vals, starting_weights, out=True)
res.summary()

Optimization terminated successfully.
         Current function value: 1330.647001
         Iterations: 31
         Function evaluations: 86
         Gradient evaluations: 86
Optimization terminated successfully.
         Current function value: 16.923536
         Iterations: 31
         Function evaluations: 36
         Gradient evaluations: 36
Optimization terminated successfully.
         Current function value: 14.018511
         Iterations: 26
         Function evaluations: 33
         Gradient evaluations: 33
Optimization terminated successfully.
         Current function value: 13.669793
         Iterations: 24
         Function evaluations: 28
         Gradient evaluations: 28
Optimization terminated successfully.
         Current function value: 13.653674
         Iterations: 24
         Function evaluations: 28
         Gradient evaluations: 28
Optimization terminated successfully.
         Current function value: 13.659066
         Iterations: 24
         Function evaluation

<class 'statsmodels.iolib.summary.Summary'>
"""
                                                   gmm Results                                                    
==================================================================================================================
Dep. Variable:     ['y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y7', 'y8', 'y9']   Hansen J:                        3990.
Model:                                                                gmm   Prob (Hansen J):                  0.00
Method:                                                               GMM                                         
Date:                                                    Thu, 20 Oct 2022                                         
Time:                                                            21:21:57                                         
No. Observations:                                                     292                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0            1.8151      0.183      9.914      0.000       1.456       2.174
p 1            1.7917      0.184      9.740      0.000       1.431       2.152
p 2            1.6214      0.175      9.239      0.000       1.277       1.965
p 3            1.8798      0.205      9.170      0.000       1.478       2.282
p 4            1.6017      0.177      9.040      0.000       1.254       1.949
p 5            2.1831      0.232      9.395      0.000       1.728       2.639
p 6            1.4239      0.104     13.646      0.000       1.219       1.628
p 7            1.2958      0.139      9.298      0.000       1.023       1.569
p 8            1.2148      0.099     12.232      0.000       1.020       1.409
p 9            1.4674      0.127     11.533      0.000       1.218       1.717
p10            1.5227      0.151     10.099      0.000       1.227       1.818
p11            1.9323      0.158     12.240      0.000       1.623       2.242
p12            1.0117      0.028     35.820      0.000       0.956       1.067
p13            0.8664      0.063     13.766      0.000       0.743       0.990
p14            1.0102      0.071     14.285      0.000       0.872       1.149
p15            1.0670      0.069     15.449      0.000       0.932       1.202
p16            1.3310      0.177      7.535      0.000       0.985       1.677
p17            1.6173      0.135     11.993      0.000       1.353       1.882
p18           -1.4911      1.006     -1.482      0.138      -3.463       0.481
p19            3.1954      0.801      3.989      0.000       1.625       4.766
==============================================================================
"""

In [73]:
premia = step1_vals[-2:]
tvals = step1_tvals[-2:]
pvals = step1_pvals[-2:]

premia = Series(premia, index=['NdrMinus', 'Ncf'])
tvals = Series(tvals, index=['NdrMinus', 'Ncf'])
pvals = Series(pvals, index=['NdrMinus', 'Ncf'])

print('Annualized Risk Premia')
print(premia*4)
print('T-stats')
print(tvals)
print('P-values')
print(pvals)

Annualized Risk Premia
NdrMinus    -5.964487
Ncf         12.781764
dtype: float64
T-stats
NdrMinus   -1.482298
Ncf         3.988915
dtype: float64
P-values
NdrMinus    0.138261
Ncf         0.000066
dtype: float64


In [74]:
# overidentification test
val, pval, dof = res.jtest()

print(f"Hansen J test with {dof} degrees of freedom returned {round(val,4)}, at {round(pval,4)} significance")

Hansen J test with 7 degrees of freedom returned 3989.7539, at 0.0 significance


In [75]:
# get weights from step 1
step1_weights = np.cov(step1_moments.T)

# run a stage 2 gmm
GMM2 = gmm(endog=excessRet, 
          exog=factors,
          instrument=inst, 
          k_moms=27, 
          k_params=3)
res = GMM2.fit(start_params=step1_vals, inv_weights=inv(step1_weights))
step2_vals = res.params
step2_tvals = res.tvalues
step2_pvals = res.pvalues
_, step2_moments = GMM1.gmmobjective(step2_vals, inv(step1_weights), out=True)
res.summary()

         Current function value: 80043.472569
         Iterations: 31
         Function evaluations: 94
         Gradient evaluations: 83
Optimization terminated successfully.
         Current function value: 16.904722
         Iterations: 30
         Function evaluations: 36
         Gradient evaluations: 36
Optimization terminated successfully.
         Current function value: 14.032426
         Iterations: 26
         Function evaluations: 32
         Gradient evaluations: 32
Optimization terminated successfully.
         Current function value: 13.669488
         Iterations: 25
         Function evaluations: 29
         Gradient evaluations: 29
Optimization terminated successfully.
         Current function value: 13.653456
         Iterations: 24
         Function evaluations: 28
         Gradient evaluations: 28
Optimization terminated successfully.
         Current function value: 13.659069
         Iterations: 24
         Function evaluations: 29
         Gradient evaluations: 

<class 'statsmodels.iolib.summary.Summary'>
"""
                                                   gmm Results                                                    
==================================================================================================================
Dep. Variable:     ['y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y7', 'y8', 'y9']   Hansen J:                        3990.
Model:                                                                gmm   Prob (Hansen J):                  0.00
Method:                                                               GMM                                         
Date:                                                    Thu, 20 Oct 2022                                         
Time:                                                            21:22:03                                         
No. Observations:                                                     292                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0            1.8151      0.183      9.914      0.000       1.456       2.174
p 1            1.7917      0.184      9.740      0.000       1.431       2.152
p 2            1.6214      0.175      9.239      0.000       1.277       1.965
p 3            1.8798      0.205      9.170      0.000       1.478       2.282
p 4            1.6017      0.177      9.040      0.000       1.254       1.949
p 5            2.1831      0.232      9.395      0.000       1.728       2.639
p 6            1.4239      0.104     13.646      0.000       1.219       1.628
p 7            1.2958      0.139      9.298      0.000       1.023       1.569
p 8            1.2148      0.099     12.232      0.000       1.020       1.409
p 9            1.4674      0.127     11.533      0.000       1.218       1.717
p10            1.5227      0.151     10.099      0.000       1.227       1.818
p11            1.9323      0.158     12.240      0.000       1.623       2.242
p12            1.0117      0.028     35.820      0.000       0.956       1.067
p13            0.8664      0.063     13.766      0.000       0.743       0.990
p14            1.0102      0.071     14.285      0.000       0.872       1.149
p15            1.0670      0.069     15.449      0.000       0.932       1.202
p16            1.3310      0.177      7.535      0.000       0.985       1.677
p17            1.6173      0.135     11.993      0.000       1.353       1.882
p18           -1.4911      1.006     -1.482      0.138      -3.463       0.481
p19            3.1954      0.801      3.989      0.000       1.625       4.766
==============================================================================
"""

In [76]:
G = GMM2.gmm_G(step2_vals)
S = np.cov(step2_moments.T)
vcv = inv(G @ inv(S) @ G.T)/T
premia_vcv = vcv[-2:,-2:]
premia_stderr = np.diag(premia_vcv)

In [77]:
premia = step2_vals[-2:]
tvals = step2_tvals[-2:]
pvals = step2_pvals[-2:]

premia = Series(premia, index=['NdrMinus', 'Ncf'])
tvals = Series(tvals, index=['NdrMinus', 'Ncf'])
pvals = Series(pvals, index=['NdrMinus', 'Ncf'])
premia_stderr = Series(premia_stderr,index=['NdrMinus', 'Ncf'])

print('Annualized Risk Premia')
print(premia*4)
print('T-stats')
print(premia_stderr)
print('P-values')
print(pvals)

Annualized Risk Premia
NdrMinus    -5.964489
Ncf         12.781762
dtype: float64
T-stats
NdrMinus    1.513936
Ncf         1.301730
dtype: float64
P-values
NdrMinus    0.138261
Ncf         0.000066
dtype: float64


In [78]:
# overidentification test
val, pval, dof = res.jtest()

print(f"Hansen J test with {dof} degrees of freedom returned {round(val,4)}, at {round(pval,4)} significance")

Hansen J test with 7 degrees of freedom returned 3989.7557, at 0.0 significance
